In [0]:
# Use Spark SQL to run a SQL query that selects all columns from the 'friends_data_csv' table in the 'default' database
df = spark.sql("select * from default.friends_data_csv")

# Display the first few rows of the DataFrame to verify that the data has been loaded correctly
df.show()


+---+-----+---+--------+------+------------+
| id| name|age|start_id|end_id|relationship|
+---+-----+---+--------+------+------------+
|  1|Alice| 30|       1|     2|      FRIEND|
|  2|  Bob| 25|       1|     3|      FRIEND|
|  3|Carol| 35|       2|     4|      FRIEND|
|  4| Dave| 40|       3|     5|      FRIEND|
|  5|  Eve| 28|       4|     6|      FRIEND|
|  6|Frank| 33|       5|     7|      FRIEND|
|  7|Grace| 38|       6|     7|      FRIEND|
+---+-----+---+--------+------+------------+



Create Nodes for our graph:

In [0]:
# Define a Cypher query to merge or update a node in Neo4j database with person information
Person_Query = """
MERGE (person:Person {id: event.id})  # Merge or create a node labeled 'Person' with id matching event.id
SET person.name = event.name,         # Set properties of the 'person' node: name, age, start_id, and end_id
    person.age = event.age,
    person.start_id = event.start_id,
    person.end_id = event.end_id
"""


In [0]:
print(Person_Query)


MERGE (person:Person {id: event.id})  # Merge or create a node labeled 'Person' with id matching event.id
SET person.name = event.name,         # Set properties of the 'person' node: name, age, start_id, and end_id
    person.age = event.age,
    person.start_id = event.start_id,
    person.end_id = event.end_id



In [0]:
# Write DataFrame 'df' to Neo4j using the Neo4j Spark Connector
df.write.format("org.neo4j.spark.DataSource")\

    # Neo4j Bolt URL
    .option("url", "bolt://34.237.142.211:7687")\

    # Authentication type and credentials
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "pits-diameters-lace")\

    # Neo4j database name
    .option("database", "friends-db")\

    # Cypher query to execute (Person_Query defined earlier)
    .option("query", Person_Query)\

    # Mode: Overwrite existing data in Neo4j
    .mode("Overwrite")\

    # Save the DataFrame to Neo4j
    .save()


Create Relationship between Person nodes

In [0]:

Person_Relationship="""
MATCH (a:Person {id: event.start_id})
MATCH (b:Person {id: event.end_id})
CREATE (a)-[:FRIEND]->(b)"""

In [0]:
print(Person_Relationship)


MATCH (a:Person {id: event.start_id})
MATCH (b:Person {id: event.end_id})
CREATE (a)-[:FRIEND]->(b)


In [0]:
# Write to Neo4j using the Neo4j Spark Connector
df.write.format("org.neo4j.spark.DataSource")\

    # Neo4j Bolt URL
    .option("url", "bolt://34.237.142.211:7687")\

    # Authentication type and credentials
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "pits-diameters-lace")\

    # Neo4j database name
    .option("database", "friends-db")\

    # Cypher query to execute (Person_Relationship defined earlier)
    .option("query", Person_Relationship)\

    # Mode: Overwrite existing data in Neo4j
    .mode("Overwrite")\

    # Save the DataFrame to Neo4j
    .save()


Read Data from Neo4j Database

In [0]:
# Define a Cypher query to read data from Neo4j
read_query = """
MATCH (p1:Person)-[r:FRIEND]->(p2:Person)  # Match nodes labeled 'Person' connected by 'FRIEND' relationship
RETURN p1, p2                              # Return nodes p1 and p2
"""


In [0]:
# Print the Cypher query string to verify or debug
print(read_query)


MATCH (p1:Person)-[r:FRIEND]->(p2:Person) RETURN p1,p2



In [0]:
# Read data from Neo4j using the Neo4j Spark Connector
read_df = spark.read.format("org.neo4j.spark.DataSource")\

    # Neo4j Bolt URL
    .option("url", "bolt://34.237.142.211:7687")\

    # Authentication type and credentials
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "pits-diameters-lace")\

    # Neo4j database name
    .option("database", "friends-db")\

    # Cypher query to execute (read_query defined earlier)
    .option("query", read_query)\

    # Load data into a DataFrame
    .load()


+---------------------------------+---------------------------------+
|p1                               |p2                               |
+---------------------------------+---------------------------------+
|{4, [Person], Frank, 33, 6, 5, 7}|{0, [Person], Grace, 38, 7, 6, 7}|
|{1, [Person], Eve, 28, 5, 4, 6}  |{0, [Person], Grace, 38, 7, 6, 7}|
|{2, [Person], Carol, 35, 3, 2, 4}|{1, [Person], Eve, 28, 5, 4, 6}  |
|{3, [Person], Alice, 30, 1, 1, 2}|{2, [Person], Carol, 35, 3, 2, 4}|
|{5, [Person], Dave, 40, 4, 3, 5} |{4, [Person], Frank, 33, 6, 5, 7}|
|{6, [Person], Bob, 25, 2, 1, 3}  |{5, [Person], Dave, 40, 4, 3, 5} |
|{3, [Person], Alice, 30, 1, 1, 2}|{6, [Person], Bob, 25, 2, 1, 3}  |
+---------------------------------+---------------------------------+

